##### Copyright 2020 The OpenFermion Developers

# Introduction to FQE


OpenFermion-FQE is an emulator for quantum computing specialized for simulations of Fermionic many-body problems,
where FQE stands for 'Fermionic quantum emulator.' By focusing on Fermionic physics,
OpenFermion-FQE realizes far more efficient emulation of quantum computing than generic quantum emulators such as Cirq,
both in computation and memory costs; the speed-up and improved memory footprint originate from the use of the spin and
number symmetries and highly optimized special algorithms.

The examples of the problems that can be simulated by OpenFermion-FQE include those in molecular electronic structure,
condensed matter physics, and nuclear physics.

The initial version of OpenFermion-FQE has been developed in collaboration between QSimulate (https://qsimulate.com)  and Google Quantum AI. The source code is found in the GitHub repository (https://github.com/quantumlib/OpenFermion-FQE).


This tutorial will describe the data structures and conventions of the library.



In [ ]:
import fqe
import numpy

## The FQE Wavefunction

The `Wavefunction` is an interface to the objects that hold the actual wavefunction data.  As mentioned the wavefunction is partitioned into sectors with fixed particle and Sz quantum number.  This partitioning information is the necessary information for initializing a `Wavefunciton` object.  As an example, we consider the initialization of a wavefunction with four spatial orbitals, four electrons, and different Sz expectation values.  The `Wavefunction` object takes a list of these sectors `[[n_electrons, sz, n_orbits]]`.

In [ ]:
wfn = fqe.Wavefunction([[4, 4, 4], [4, 2, 4], [4, 0, 4], [4, -2, 4], [4, -4, 4]])

This command initializes a wavefunction with the following block structure

![Image of wf sectors](./wf_init_sectors.png)

Each sector corresponds to a set of bit strings $$\vert I \rangle = \vert I_{\alpha}I_{\beta}\rangle$$ that encode a fixed particle number and fixed $Sz$ expectation. The coefficients associated with the bitstrings in these sectors are formed into matrices.  This helps with efficient vectorized computations. The row-index of the array corresponds to the $\alpha$ spin-orbital number occupation index and the column-index corresponds to the $\beta$-strings.  The `Waveefunction` object provides tools to access sectors or perform basic mathematical operations on this *vector*.  

## Initialization of wave functions
FQE wave functions can be initialized by calling the constructor directly. fqe.get_wavefunction and get_wavefunction_mutltiple are utility wrappers that provide wrappers for common initialization motifs. e.g. get_spin_nonconserving_wavefunction will build a wavefunction that permits spin breaking Operators to be applied while get_wavefunction_multiple will return a list of initialize wavefunctions

In [ ]:
wfFqe = fqe.Wavefunction([[4, -2, 10]], broken=None)
wfFqe = fqe.get_wavefunction(4, -2, 10)

In [ ]:
n = 3
m = -2
wfFQE = fqe.Wavefunction([[n, m, 10], [n+1, m+2, 10]], broken=None)
wfFQE = fqe.get_wavefunction_multiple([[4, 0, 10], [5, -5, 10]])

In [ ]:
wfFQE = fqe.get_number_conserving_wavefunction(2, 4)
print("Empty Initialization")
wfFQE.print_wfn()

print()
print("Random Initialization")
wfFQE.set_wfn(strategy='random')
wfFQE.print_wfn()

We also provide functionality for converting wavefunctions to and from cirq.  An important thing to note is the ordering of the $\alpha$ and $\beta$ strings in the converted wavefunctions.  The FQE uses the OpenFermion convention of interleaved $\alpha$ and $\beta$ orbitals.  Thus when converting to cirq we first convert each bit string into an OpenFermion operator and then call normal ordering.

In [ ]:
nqubit = 4

cirq_wfn = numpy.random.rand(2**nqubit).astype(numpy.complex128) + numpy.random.rand(2**nqubit).astype(numpy.complex128)*1.j
norm = numpy.sqrt(numpy.vdot(cirq_wfn, cirq_wfn))
numpy.divide(cirq_wfn, norm, out=cirq_wfn)
wfn_from_cirq = fqe.from_cirq(cirq_wfn, 0.0001)

print('Inital wavefunction is \n {} \n with norm {}'.format(cirq_wfn, numpy.vdot(cirq_wfn, cirq_wfn)))
wfn_from_cirq.print_wfn()

print()
print("FQE wf to cirq")
print(fqe.to_cirq(wfn_from_cirq))

Users can access the wavefunction through the `get_sector` method.  This returns the entire matrix of data representing the specified sector of the wavefunction.  For example, we can grab the sector corresponding to $n = 2$ and $sz = 0$

In [ ]:
interesting_states = wfn_from_cirq.get_coeff((2, 0))
print(interesting_states)

### 2.A.5 Printing/Saving

Printing is currently availible as alpha beta strings followed by the coefficient as well as orbital occupation representation.

In [ ]:
print('String forrmatting')
wfn_from_cirq.print_wfn(fmt='str')
print('\n Occupation forrmatting')
wfn_from_cirq.print_wfn(fmt='occ')

# Action on Wavefunctions: Fermionic algebra operations and their unitaries on the state

FermionOperators can be directly passed in to create a newwavefunction based on application of the operators.  The FermionOperators are parsed according to the interleaved $\alpha$ $\beta$ indexing of the spin-orbitals.  This means that odd index FermionOperators correspond to $\beta$-spin orbitals and even are $\alpha$-spin orbitals.  

Sharp Edge:

The user must be careful to not break the symmetry of the wavefunction.  If a request to apply an operator to a state takes the wavefunction outside of the specified symmetry sector the FQE will not execute the command.  Effectively, the FQE requires the user to have more knowledge of what type of operations their `Wavefunction` object can support.

In [ ]:
from openfermion import FermionOperator, hermitian_conjugated
ops = FermionOperator('2^ 0', 1.2)
new_wfn = wfn_from_cirq.apply(ops + hermitian_conjugated(ops))

## unitary operations:

Any simulator backend must be able to perform unitary evolution on a state.  The FQE accomplishes this by implementing code for evolving a state via the action of a unitary generated by fermionic generators.  Given $g$ is a fermion operator the unitary $$e^{-i (g + g^{\dagger})}$$ can be applied to the state.  It can be shown that this evolution operator can be rewritten as 
$$ e^{-i(g + g^{\dagger})\epsilon } = \cos\left(\epsilon\right) \mathbb{I}_{s}(gg^{\dagger}) +  \cos\left(\epsilon\right) \mathbb{I}_{s}(g^{\dagger}g) - i\sin\left(\epsilon\right) \left(g + g^{\dagger}\right) \left[\mathbb{I}_{s}(gg^{\dagger}) + \mathbb{I}_{s}(g^{\dagger}g)\right] + \mathbb{I}_{!s}$$
The $\mathbb{I}_{!s}$ is for setting the coefficients of the unitary that are not in the subspace $\mathcal{H}_{s} \subset \mathcal{H}$ where $gg^{\dagger}$ is 0.   

The user can specify a fermionic monomial in OpenFermion and use the `time_evolve` method of the `Wavefunction` object to call the evolution. All the rules for preserving symmetries must be maintained as before.

In [ ]:
i, j, theta = 0, 1, numpy.pi / 3
op = FermionOperator(((2 * i, 1), (2 * j, 0)), coefficient=-1j * theta) + FermionOperator(((2 * j, 1), (2 * i, 0)), coefficient=1j * theta)
fqe_wfn = wfn_from_cirq.time_evolve(1.0, op)

fqe_wfn.print_wfn()

In other tutorials we will do a deeper dive into supported time-evolution operations.  To serve a full functioning time-evolution platform the FQE also implements arbitrary time-evolution of full Hamiltonian operators consisting of sums of non-commuting terms.  The Taylor and Chebyshev expansion methods are used to do the exact time evolution.  